In [1]:
import matplotlib.pyplot as plt
import numpy as np
from utils.objects import GenerationClass
from utils.config import color_maps, ycb_heights
import os
from glob import glob
from scipy import sparse
from PIL import Image
import json
from mani_skill2.envs.pick_and_place.pick_cube import PickCubeEnv
from mani_skill2.utils.sapien_utils import look_at
from mani_skill2.sensors.camera import CameraConfig
from mani_skill2.utils.registration import register_env
from pathlib import Path
import gymnasium as gym
from sapien.core import Pose
import numpy as np
from transforms3d.euler import euler2quat, quat2euler
from utils.config import ycb_heights, color_maps
from glob import glob
from scipy import sparse
from PIL import Image
import json
import os

from IPython.display import clear_output
# save_dir = "../final_data/"
# move_one_dir = os.path.join(save_dir, "move")
# move_cube_dir = os.path.join(move_one_dir, "cube")
# move_ycb_dir = os.path.join(move_one_dir, "ycb")
# os.makedirs(move_one_dir, exist_ok=True)
# os.makedirs(move_cube_dir, exist_ok=True)
# os.makedirs(move_ycb_dir, exist_ok=True)

In [2]:
model_paths = sorted(glob("../data/mani_skill2_ycb/models/*"), key=lambda x: int(x.split('/')[-1][:3]))
gen = GenerationClass(model_paths)

In [4]:
# gen.generate_single_direction_cubes(1)
# gen.generate_single_direction_cubes(3)
# gen.generate_single_direction_cubes(4)

# gen.generate_in_between_cubes(3)


# gen.generate_on_another(2)
# gen.generate_on_another(3)
# gen.generate_on_another(4)



In [ ]:
N = 4

obj_cfgs, camera_cfgs = gen.cube_configs(N)

colors = list(color_maps.keys()) # 9 colors
random_scales = np.arange(3, 4) * 2

total_idx = 0

for color in colors:
    for idx in range(20):
        obj_cfgs = []
        obj_cfgs.append({
            "name": f"{color}_cube",
            "scale": np.random.choice(random_scales),
            "color": color_maps[color],
            "color_name": color,
            "static": True,
        })

        # pick 3 not in color
        not_in_colors = [c for c in colors if c != color]
        random_colors = np.random.choice(not_in_colors, 3, replace=False)

        for i in range(3):
            color = random_colors[i]
            obj_cfgs.append({
                "name": f"{color}_cube",
                "scale": np.random.choice(random_scales),
                "color": color_maps[color],
                "color_name": color,
                "static": True,
        })

        env = gen.get_env(obj_cfgs, camera_cfgs)
        gen.generate_samples(N, obj_cfgs, idx, total_idx, type="place_object_on_another")
        total_idx += 1
        clear_output(wait=True)
        print("Done", total_idx, end="\r")

In [ ]:
colors = np.random.choice(list(color_maps.keys()), 4, replace=False)
random_scales = np.random.permutation(np.arange(3, 7))

configs= [{
    "name": "cube", 
    "scale": random_scales[i],
    "color": color_maps[colors[i]],
    "color_name": colors[i],
    "static": True,
} for i in range(4)]

env = get_env(configs, {"p": [-1, 0, 0.5], "fov": 1.2})
initialize_positions(configs, min_dist=min_distance)

fig, axs = plt.subplots(1, 2)
obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
axs[0].imshow(obs['image']['base_camera']['rgb'])


order_list = [configs[i]["color_name"] for i in range(len(env.unwrapped.objs))][::-1]
order_by_color(env, order_list, colors)
obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
axs[1].imshow(obs['image']['base_camera']['rgb'])
plt.suptitle(f"Ordered by {order_list}")
plt.show()

In [ ]:
colors = np.random.choice(list(color_maps.keys()), 4, replace=False)
random_scales = np.random.permutation(np.arange(3, 7))
configs= [{
    "name": "cube", 
    "scale": random_scales[i],
    "color": color_maps[colors[i]],
    "color_name": colors[i],
    "static": True,
} for i in range(4)]

env = get_env(configs, {"p": [-1, 0, 0.5], "fov": 1.2})
initialize_positions(configs, min_dist=min_distance)

fig, axs = plt.subplots(1, 2)
obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
axs[0].imshow(obs['image']['base_camera']['rgb'])
order_by_size(env, configs)

obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
axs[1].imshow(obs['image']['base_camera']['rgb'])
plt.suptitle(f"Ordered by size")
plt.show()

In [ ]:
def get_ycb_order_config(model_paths):
    model_paths = sorted(glob("../data/mani_skill2_ycb/models/*"), key=lambda x: int(x.split('/')[-1][:3]))
    configs = [
        {
            "name": model_paths[i],
            "scale": 2,
            "static": True,
        }
        for i in np.random.choice(np.arange(len(model_paths)), 4, replace=False).astype(int)
    ]
    camera_cfgs = {"p": [-1, 0, 0.5], "fov": 1.2}
    return configs, camera_cfgs

configs, camera_cfgs = get_ycb_order_config(model_paths)
env = get_env(configs, camera_cfgs)
initialize_positions(configs, min_dist=min_distance)

fig, axs = plt.subplots(1, 2)
obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
axs[0].imshow(obs['image']['base_camera']['rgb'])

name_list = [c["name"] for c in configs][::-1]
name_list_str = [c.split('/')[-1] for c in name_list]
order_by_name(env, name_list, configs)
obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
axs[1].imshow(obs['image']['base_camera']['rgb'])


plt.suptitle(f"Ordered by {name_list_str}")
plt.show()

In [ ]:
def get_motion_configs(model_paths):
    configs = [
        {
            "name": model_paths[i],
            "scale": 2,
            "static": False,
        }
        for i in np.random.choice(np.arange(len(model_paths)), 4, replace=False).astype(int)
    ]
    return configs

In [ ]:
# motion
model_paths = sorted(glob("../data/mani_skill2_ycb/models/*"), key=lambda x: int(x.split('/')[-1][:3]))
configs = get_motion_configs(model_paths)
env = get_env(configs)

initialize_positions(configs, min_dist=min_distance)
random_force = np.random.choice([-250, 250], 3)


random_force = get_random_force()

random_force[2] = 0
motion_map = {
    "forward": np.array([-250, 0, 0]),
    "backward": np.array([250, 0, 0]),
    "right": np.array([0, -250, 0]),
    "left": np.array([0, 250, 0]),
    "random": random_force,
    # "up": np.array([0, 0, 125]), # hard to determine between direction
}


rand_obj = np.random.choice(env.unwrapped.objs)
rand_direction = np.random.choice(list(motion_map.keys()))

[obj.set_velocity(np.array([0, 0, 0])) for obj in env.unwrapped.objs]

# rand_obj = env.objs[0]
# rand_direction = "random"
# rand_direction = "up"


for idx in range(1, 9):  # Simulate for 1 second
    move_object(rand_obj, rand_direction, motion_map, 1/idx)
    # env.obj.add_force_at_point(np.array([0, 0 , 1000]), np.array([0, 0, 0]))
    obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
    clear_output(wait=True)

    plt.imshow(obs['image']['base_camera']['rgb'])
    plt.title(f"Move {rand_obj.name} {rand_direction}")
    plt.show()
    time.sleep(0.1)

In [ ]:
model_paths = sorted(glob("../data/mani_skill2_ycb/models/*"), key=lambda x: int(x.split('/')[-1][:3]))
configs = [
    {
        "name": model_paths[i],
        "scale": 2,
        "static": False,
    }
    for i in np.random.uniform(0, len(model_paths), 4).astype(int)
]


env = get_env(configs)
initialize_positions(configs, min_dist=min_distance)

obj1 = env.unwrapped.objs[0]
obj2 = env.unwrapped.objs[1]

direction = get_object_to_object_direction(obj1, obj2)

for idx in range(1, 9):  # Simulate for 1 second
    move_object_toward_another(obj1, direction, 1/(np.sqrt(idx)))
    obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
    clear_output(wait=True)

    plt.imshow(obs['image']['base_camera']['rgb'])
    plt.show()
    time.sleep(0.1)

In [ ]:
### imported 3d objects


@register_env("CustomEnv-v0", max_episode_steps=200, override=True)
class CustomEnv(PickCubeEnv):
    def _load_actors(self):
        self._add_ground(render=self.bg_name is None)
        objs = []
        builder = self._scene.create_actor_builder()

        name = "eiffel_tower"

        filepath = f"/home/mkim/Nserver/mix-lab/maniskill2_dev/models/{name}/mesh.obj"
        scale = 0.5
        scale *= self.cube_half_size / 0.01887479572529618
        builder.add_multiple_collisions_from_file(
            filename=filepath, scale=scale, density=1000
        )
        builder.add_visual_from_file(filename=filepath, scale=scale)
        self.obj = builder.build_static(name=name)

        self.objs = objs
        self.goal_site = self._build_sphere_site(self.goal_thresh)

    def _register_cameras(self):
        pose = look_at([-1, 0, 1], [0, 0, 0])
        return CameraConfig(
            "base_camera", pose.p, pose.q, 512, 512, 1.2, 0.01, 10
        )
    

env = gym.make("CustomEnv-v0", obs_mode="rgbd", camera_cfgs={"add_segmentation": True})

obs, _ =env.reset()


env.unwrapped.get_articulations()[0].set_pose(Pose([0, 0, -2], [1, 0, 0, 0]))
# quat = euler2quat(np.random.uniform(-np.pi*2, np.pi*2), np.random.uniform(-np.pi*2, np.pi*2), np.random.uniform(-np.pi*2, np.pi*2))
quat = euler2quat(np.pi/2, 0, np.random.uniform(np.pi*3/4, np.pi*5/4))#np.random.uniform(-np.pi*2, np.pi*2))
# quat = np.array([ 0.37677702, -0.25098675, -0.80399704, -0.38553014])
env.obj.set_pose(Pose([-0.5, 0, 0.3], quat))
# for i in range(10):
obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))

img = obs['image']["base_camera"]["rgb"]
mask = obs['image']["base_camera"]["Segmentation"][..., :3].copy()
mask[mask != env.obj.id-2] = 0
mask = np.amax(mask.astype(bool), axis=2)

img[mask] += 50
# img[mask] += 25
img = np.clip(img, 0, 255)

clear_output(wait=True)
plt.imshow(img)
plt.show()